## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install scikit-learn==0.22.2.post1
!pip install flask-ngrok
!pip install face-recognition

     |████████████████████████████████| 7.1 MB 4.5 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.22.2.post1 which is incompatible.
     |████████████████████████████████| 100.1 MB 21 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=f801a3a1eecb41292055f9c9a38c1a1cc4a53e83d73cec2cecc4e80459be0839
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


SSH Configuration

In [3]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

/drive/ngrok-ssh
--2022-02-03 05:53:39--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.237.133.81, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  41.9MB/s    in 0.3s    

2022-02-03 05:53:40 (41.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [4]:
!/ngrok authtoken 1yigVfltPzIgqMmhBcdATqbWeES_2YJKH3yoA9KDk38XgdvK6

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
cd /content/drive/MyDrive/PicClickAuto

/content/drive/MyDrive/PicClickAuto


## Library

In [6]:
from flask import Flask, render_template,jsonify,request
from flask_ngrok import run_with_ngrok
from PIL import Image
from sklearn.svm import SVC
import joblib
import PIL
import io
import base64
import cv2
import numpy as np
import joblib
import dlib
import face_recognition

In [7]:
!wget --no-check-certificate \
    https://raw.githubusercontent.com/italojs/facial-landmarks-recognition/master/shape_predictor_68_face_landmarks.dat \
    -O shape_predictor_68_face_landmarks.dat

--2022-02-03 05:54:01--  https://raw.githubusercontent.com/italojs/facial-landmarks-recognition/master/shape_predictor_68_face_landmarks.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99693937 (95M) [application/octet-stream]
Saving to: ‘shape_predictor_68_face_landmarks.dat’

shape_predictor_68_ 100%[===================>]  95.08M  37.0MB/s    in 2.6s    

2022-02-03 05:54:05 (37.0 MB/s) - ‘shape_predictor_68_face_landmarks.dat’ saved [99693937/99693937]



In [8]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [9]:
file_model = "/content/drive/MyDrive/PicClickAuto/model/SVM_LINEAR_DIFF_2.pkl"
joblib_model = joblib.load(file_model)

## Function

In [10]:
def Crop_Face(image):
    '''
    Memotong gambar sesuai dengan 7 titik wajah pada face landmark
    sehingga mendapatkan area Wajah
    '''
    try:
      rect = detector(image)[0]
    except (ValueError,IndexError):
      print("Not Found Face!!")
      return image
  
    sp = predictor(image, rect)
    landmarks = np.array([[p.x, p.y] for p in sp.parts()])
    x = []
    y_alis = []
    y = []
    w = []
    h = []
    
    x.append(landmarks[1][0])
    y_alis.append(landmarks[17][1])
    y_alis.append(landmarks[18][1])
    y_alis.append(landmarks[23][1])
    y_alis.append(landmarks[24][1])
    w.append(landmarks[15][0])
    h.append(landmarks[8][1])
    y.append(min(y_alis))
    
    
    crop_img = image[y[0]:h[0], x[0]:w[0]]
  
    return crop_img

def Preprocessing (base_image,sig,size_x,size_y):
    '''
    Konversi Gray,Memotong daerah Wajah, Gaussian Blur, Resize
    '''
    global face_pre,crop_img
    if(base_image is not None):
      grey = cv2.cvtColor(base_image, cv2.COLOR_BGR2GRAY)
      crop_img = Crop_Face(grey)
      face_pre = cv2.GaussianBlur(crop_img, ksize=(0, 0), sigmaX=sig, borderType=cv2.BORDER_REPLICATE)
      face = np.double(cv2.resize(np.array(face_pre),(size_x,size_y)))
    
    return face

def Feature(flash,background):
    '''
    Ektrasi Feature Pantulan
    '''
    
    a = flash - background
    b = flash + background
    c = a/b
    trans = np.transpose(c)
    feat_vec = np.reshape(trans, (trans.size,))
    feat_vec = np.nan_to_num(feat_vec)
    return feat_vec

def ConvertToNumpy(myfile):
    imgdata = base64.b64decode(myfile)
    im = Image.open(io.BytesIO(imgdata))
    image_np = np.array(im)
    return image_np

## APP

In [11]:
import os

path_data = '/content/drive/MyDrive/PicClickAuto/data'
face_locations = []
face_encodings = []
face_names = []
known_face_encodings = []
known_face_names = []

for folder in os.listdir(path_data):
  folder_loc = os.path.join(path_data,folder)
  for filename in os.listdir(folder_loc):
    img = face_recognition.load_image_file(os.path.join(folder_loc,filename))
    face_encoding = face_recognition.face_encodings(img)[0]
    known_face_encodings.append(face_encoding)
    known_face_names.append(folder)

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/image_info",methods= ['GET'])
def image_info():
    myfile= request.args.get('myimage').split(',')

    bg_image = ConvertToNumpy(myfile[1])
    flash_image = ConvertToNumpy(myfile[3])

    bg = Preprocessing(bg_image, 5, 100, 100)
    flash = Preprocessing(flash_image, 5, 100, 100)

    pantulan = Feature(flash, bg)

    predection = joblib_model.predict([pantulan])
    #print(predection)

    if predection[0] == 1:
      FAS = 'real'
    else:
      FAS = 'spoof'

    small_frame = cv2.resize(flash_image, (0, 0), fx=0.25, fy=0.25)
    face_locations = face_recognition.face_locations(small_frame)
    face_encodings = face_recognition.face_encodings(small_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)

        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        # Or instead, use the known face with the smallest distance to the new face
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)

        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        face_names.append(name)

    return jsonify(FAS=FAS, face_names=face_names)

@app.route("/")
def photoClick():
    return render_template('photoclick.html')

if __name__ == '__main__':
   app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://bd85-104-154-163-81.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [03/Feb/2022 06:20:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2022 06:20:53] "GET /static/webcam.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2022 06:21:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2022 06:21:07] "GET /static/webcam.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2022 06:21:08] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [03/Feb/2022 06:21:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2022 06:21:14] "GET /static/webcam.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2022 06:21:15] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [03/Feb/2022 06:21:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2022 06:21:45] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [03/Feb/2022 06:22:02] "GET /image_info?myimage=data%3Aimage%2Fjpeg%3Bbase64%2C%2F9j%2F4AAQSkZJRgABAQAAAQABAAD%2F2wBDAAMCAgMCAgMDAwMEAwMEBQgFBQQEBQoHBwYIDAoMDAsKCwsNDhIQDQ4RDgsLEBYQERMUFRUVDA8XGBYUGBIUFRT%2F2wBDAQMEBAUEBQkFBQkUDQsNFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBT%2FwAAR

Not Found Face!!
